In [79]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from ClassificadorAlfa import ClassificadorAlfa
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(os.path.join('data', 'healthcare-dataset-stroke-data.csv'))

## Hipótese 1

Para a hipótese 1, vamos testar todos os dados contidos no DataFrame, sem alterações, porém apenas os dados que podem ser tratados como Booleanos. Vamos utilizar as variáveis categóricas para realizar o teste. A hipótese nula, no caso, seria montar um classificador que irá sempre prever os dados como sendo o mais frequente $(x)$, ou seja, será a acurácia medida ao dividir tal valor pelo total dos dados contidos no DataFrame $(N)$:

$$
Acurácia = \frac{x}{N}
$$

Portanto, vamos calcular, ao final, a acurácia da hipótese nula e verificar como a nossa hipótese, utilizando todas as variáveis categóricas, se comporta com relação a ela.

In [ ]:
# Separando target e features
try:
    df_ = df.drop(['id', 'bmi', 'age', 'avg_glucose_level'], axis=1)
except:
    pass

X = df_.drop('stroke', axis=1)
y = df_['stroke']

In [ ]:
# Obtendo variáveis categoricas
object_features = [feature for feature in X.columns if X[feature].dtype == 'O']
int_features = [feature for feature in X.columns if X[feature].dtype == 'int64']
categorical_features = object_features + int_features

In [ ]:
# Obtendo variáveis dummy
X = X[categorical_features]

X = pd.get_dummies(X)
X.hypertension = X.hypertension.astype('bool')
X.heart_disease = X.heart_disease.astype('bool')

X_names = X.columns

y = y.replace(0,-1)

X = X.to_numpy()
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
# Inicializando os parâmetros do modelo
a = np.random.randn(X_train.shape[1], 1)
b = 1.0

X_train = X_train.T
y_train = y_train.T

parametros = [a, b, X_train, y_train]
learning_rate = 0.0001
num_iteracoes = 50000

In [ ]:
classificador = ClassificadorAlfa(learning_rate, num_iteracoes, parametros)

In [ ]:
# Treinando o modelo
a, b = classificador.treinar()
a, b

In [ ]:
# Fazendo previsões
ypred = a.T @ X_test.T + b
ypred

In [ ]:
acuracia = ClassificadorAlfa.acuracia(y_test, ypred)
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

In [ ]:
# Selecionando as features mais importantes
features = X_names
importances = pd.DataFrame(data=a, index=features, columns=['importance']).sort_values(by='importance', ascending=False)
importances

In [ ]:
# Obtendo a acurácia da hipótese nula
acuracia_nula = y[y == -1].shape[0] / y.shape[0]

print(f'A acurácia da hipótese nula foi de {acuracia_nula*100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

### Utilizando um classificador de árvore de decisão

Em seguida, vamos comparar o nosso modelo com um classificador de árvore de decisão, comparando, novamente, as acurácias obtidas.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classificador = DecisionTreeClassifier(criterion='entropy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
classificador.fit(X_train, y_train)

In [ ]:
classificador.predict(X_test)

In [ ]:
acuracia = classificador.score(X_test, y_test)

In [ ]:
# Plotando a árvore de decisão
import matplotlib.pyplot as plt
plt.figure(figsize=(30, 30))
plot_tree(classificador, filled=True, rounded=True, class_names=['Não teve AVC', 'Teve AVC'], feature_names=X_names, fontsize=15, node_ids=False)
plt.show()

In [ ]:
# Obtendo as features mais importantes
importances = pd.DataFrame(data=classificador.feature_importances_, index=X_names, columns=['importance']).sort_values(by='importance', ascending=False)
importances

In [ ]:
print(f'A acurácia da hipótese nula foi de {acuracia_nula*100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

Como podemos observar com as acurácias obtidas, todas ficaram rondando os 95% nos testes. Como a grande maioria dos dados na base são de pessoas que não tiveram AVCs, o classificador da hipótese nula sempre terá uma acurácia muito alta, visto que ele sempre irá prever que a pessoa não teve AVC. Assim, o nosso modelo não funciona de maneira significativamente melhor que a hipótese nula, assim como o modelo de árvore de decisão. Na nossa próxima hipótese, tentaremos realizar o classificador, trabalhando com uma base de dados reduzida, para tentarmos verificar uma mudança mais significativa na acurácia.

## Hipótese 2

Para nossa próxima hipótese, manteremos o cálculo da acurácia da hipótese nula, porém obteremos um subset, dentro de nossa base de dados, com valores balanceados de pessoas com e sem AVCs, ou seja, teremos uma base de dados com 50% de pessoas que tiveram AVCs e 50% de pessoas que não tiveram AVCs. Assim, poderemos verificar se o nosso modelo consegue prever melhor os casos de AVCs, visto que teremos uma base de dados mais balanceada. Nesse caso, a acurácia de nosso classificador nulo será de 50%.

In [ ]:
# Preparando os dados
try:
    df_ = df.drop(['id', 'bmi', 'age', 'avg_glucose_level'], axis=1)
except:
    pass

In [ ]:
# Separando a base de dados balanceada
df_stroke = df_[df_.stroke == 1]
df_no_stroke = df_[df_.stroke == 0]
# Obtendo um sample aleatório da base de dados sem AVCs
df_no_stroke = df_no_stroke.sample(n=df_stroke.shape[0], random_state=42)
# Concatenando os dados
df_ = pd.concat([df_stroke, df_no_stroke])

In [ ]:
# Separando os dados
X = df_.drop('stroke', axis=1)
y = df_.stroke

In [ ]:
# Obtendo variáveis categoricas
object_features = [feature for feature in X.columns if X[feature].dtype == 'O']
int_features = [feature for feature in X.columns if X[feature].dtype == 'int64']
categorical_features = object_features + int_features

In [ ]:
# Obtendo variáveis dummy
X = X[categorical_features]

X = pd.get_dummies(X)
X.hypertension = X.hypertension.astype('bool')
X.heart_disease = X.heart_disease.astype('bool')

X_names = X.columns

y = y.replace(0,-1)

X = X.to_numpy()
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
# Inicializando os parâmetros do modelo
a = np.random.randn(X_train.shape[1], 1)
b = 1.0

X_train = X_train.T
y_train = y_train.T

parametros = [a, b, X_train, y_train]
learning_rate = 0.0001
num_iteracoes = 50000

In [ ]:
classificador = ClassificadorAlfa(learning_rate, num_iteracoes, parametros)

In [ ]:
# Treinando o modelo
a, b = classificador.treinar()
a, b

In [ ]:
# Fazendo previsões
ypred = a.T @ X_test.T + b
ypred

In [ ]:
acuracia = ClassificadorAlfa.acuracia(y_test, ypred)
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

In [ ]:
# Selecionando as features mais importantes
features = X_names
importances = pd.DataFrame(data=a, index=features, columns=['importance']).sort_values(by='importance', ascending=False)
importances

In [ ]:
# Obtendo a acurácia da hipótese nula
acuracia_nula = y[y == -1].shape[0] / y.shape[0]

print(f'A acurácia da hipótese nula foi de {acuracia_nula*100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

### Utilizando um classificador de árvore de decisão

Em seguida, vamos comparar o nosso modelo com um classificador de árvore de decisão, comparando, novamente, as acurácias obtidas.

In [ ]:
classificador = DecisionTreeClassifier(criterion='entropy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
classificador.fit(X_train, y_train)

In [ ]:
classificador.predict(X_test)

In [ ]:
acuracia = classificador.score(X_test, y_test)

In [ ]:
# Plotando a árvore de decisão
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 30))
plot_tree(classificador, filled=True, rounded=True, class_names=['Não teve AVC', 'Teve AVC'], feature_names=X_names,
          fontsize=15, node_ids=False)
plt.show()

In [ ]:
# Obtendo as features mais importantes
importances = pd.DataFrame(data=classificador.feature_importances_, index=X_names, columns=['importance']).sort_values(
    by='importance', ascending=False)
importances

In [ ]:
print(f'A acurácia da hipótese nula foi de {acuracia_nula * 100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia * 100:.2f}%')

Na nossa segunda hipótese, a diferença entre os modelos e a hipótese nula foi mais significativa. Obtivemos, como descrito anteriormente, uma base de dados balanceada, com 50% dos dados de pessoas que tiveram AVCs e 50% que não o tiveram. Assim, a acurácia nula da base de dados permaneceu em 50%. O modelo `ClassificadorAlfa` obteve uma acurácia de pouco mais de 59%, se tratando de uma diferença mais significativa. O classificador de árvore de decisão manteve uma acurácia de aproximadamente 53%, o que também é uma diferença significativa em relação à hipótese nula, mesmo que não tão significativa quanto a diferença obtida pelo `ClassificadorAlfa`. Em seguida, vamos analisar a nossa terceira hipótese.

## Hipótese 3

Em nossa terceira hipótese, vamos utilizar apenas os dados relacionados à saúde dos indivíduos, desconsiderando fatores como trabalho, casamento e moradia. A ideia é verificar se os dados relacionados à saúde presentes no dataset são suficientes para prever se uma pessoa teve ou não um AVC.

In [1]:
# Obtendo os dados
try:
    df_ = df.drop(['id', 'ever_married', 'work_type', 'Residence_type'], axis=1)
except:
    pass

In [ ]:
# Criando clusters para as variáveis contínuas (clusters obtidos via ChatGPT)
df_['age'] = pd.cut(df.age, bins=[0, 13, 20, 40, 60, 100], labels=['Child', 'Teenager', 'Young-adult', 'Middle-aged', 'Elder'])

df_['bmi'] = pd.cut(df.bmi, bins=[0, 16.5, 25, 30, 40, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese', 'Morbidly-obese'])

df_['avg_glucose_level'] = pd.cut(df.avg_glucose_level, bins=[0, 90, 120, 140, 180, 280], labels=['Low', 'Normal', 'High-normal', 'Mildly-high', 'Severely-high'])

In [ ]:
# Separando a base de dados balanceada
df_stroke = df_[df_.stroke == 1]
df_no_stroke = df_[df_.stroke == 0]
# Obtendo um sample aleatório da base de dados sem AVCs
df_no_stroke = df_no_stroke.sample(n=df_stroke.shape[0], random_state=42)
# Concatenando os dados
df_ = pd.concat([df_stroke, df_no_stroke])

In [ ]:
# Separando os dados
X = df_.drop('stroke', axis=1)
y = df_.stroke

In [ ]:
# Obtendo variáveis categoricas
object_features = [feature for feature in X.columns if X[feature].dtype == 'O']
int_features = [feature for feature in X.columns if X[feature].dtype == 'int64']
categorical_features = object_features + int_features

In [ ]:
# Obtendo variáveis dummy
X = X[categorical_features]

X = pd.get_dummies(X)
X.hypertension = X.hypertension.astype('bool')
X.heart_disease = X.heart_disease.astype('bool')

X_names = X.columns

y = y.replace(0, -1)

X = X.to_numpy()
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
# Inicializando os parâmetros do modelo
a = np.random.randn(X_train.shape[1], 1)
b = 1.0

X_train = X_train.T
y_train = y_train.T

parametros = [a, b, X_train, y_train]
learning_rate = 0.0001
num_iteracoes = 50000

In [ ]:
classificador = ClassificadorAlfa(learning_rate, num_iteracoes, parametros)

In [ ]:
# Treinando o modelo
a, b = classificador.treinar()
a, b

In [ ]:
# Fazendo previsões
ypred = a.T @ X_test.T + b
ypred

In [ ]:
acuracia = ClassificadorAlfa.acuracia(y_test, ypred)
print(f'A acurácia do modelo foi de {acuracia * 100:.2f}%')

In [ ]:
# Selecionando as features mais importantes
features = X_names
importances = pd.DataFrame(data=a, index=features, columns=['importance']).sort_values(by='importance', ascending=False)
importances

In [ ]:
# Obtendo a acurácia da hipótese nula
acuracia_nula = y[y == -1].shape[0] / y.shape[0]

print(f'A acurácia da hipótese nula foi de {acuracia_nula * 100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia * 100:.2f}%')

### Utilizando o classificador de árvore de decisão

In [ ]:
classificador = DecisionTreeClassifier(criterion='entropy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
classificador.fit(X_train, y_train)

In [ ]:
classificador.predict(X_test)

In [ ]:
acuracia = classificador.score(X_test, y_test)

In [ ]:
# Plotando a árvore de decisão
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 30))
plot_tree(classificador, filled=True, rounded=True, class_names=['Não teve AVC', 'Teve AVC'], feature_names=X_names,
          fontsize=15, node_ids=False)
plt.show()

In [ ]:
# Obtendo as features mais importantes
importances = pd.DataFrame(data=classificador.feature_importances_, index=X_names, columns=['importance']).sort_values(
    by='importance', ascending=False)
importances

In [ ]:
print(f'A acurácia da hipótese nula foi de {acuracia_nula * 100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia * 100:.2f}%')

Nessa análise, obtivemos dados de acurácia semelhantes aos testes anteriores. Isso indica que os dados relacionados à saúde são suficientes para prever se uma pessoa teve ou não um AVC. Ademais, ambos os classificadores mostraram, mesmo que não na mesma ordem, que as variáveis mais importantes para a decisão são as relacionadas à hipertensão, presença de doenças cardíacas e uso de cigarros.

## Hipótese 4

Para finalizar, vamos utilizar apenas as variáveis mais relevantes que foram levantadas por nossos classificadores para realizar as análises.